In [2]:
##Title Imports
from random import randint, sample, random, uniform
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm


In [3]:
class Particula():
  
  def __init__(self, position, velocity, value, pBest):
    self.position = position
    self.valuePBest = value
    self.PBest = pBest
    self.velocity = velocity
    self.indiceE = 0
    self.indiceD = 0
    
  def updateIndice(self, indiceE, indiceD):
    self.indiceE = indiceE
    self.indiceD = indiceD
    
  def updateVelocity(self, velocity):
    self.velocity = velocity
  
  def updatePBest(self, PBest):
    self.PBest = PBest

  def updateValuePBest(self, valuePBest):
    self.valuePBest = valuePBest
    
  def updatePosition(self, position):
    self.position = position

In [4]:
##Title Functions CEC 2013

def sphere(vetor):
  resultado = 0
  
  for i in vetor:
    resultado += i**2
  
  return resultado

def rastrigin(vetor):
  resultado = 0
  
  for i in vetor:
    numero = 2*3.1415*i
    p = (numero/180)*math.pi
    resultado+= (i**2) - (10 * math.cos(p)) + 10
    
  return resultado

def rosenbrock(vetor):
  resultado = 0
  
  for i in range(0,(len(vetor)-1)):
    resultado += 100*(vetor[i+1] - vetor[i]**2)**2 + (vetor[i] - 1)**2
    
  return resultado


## FUNÇÕES DE VELOCIDADE

def calculateVelocity(velocity, c1, c2, w, r1, r2, PBest, GBest, position, clerc, feasibleBounds1, feasibleBounds2):
  resultado = []
  for i in range(0,len(velocity)):
    if(clerc):
      vi = w*(velocity[i] + c1*r1*(PBest[i] - position[i]) + c2*r2*(GBest[i] - position[i]))
    else:
      vi = w*velocity[i] + c1*r1*(PBest[i] - position[i]) + c2*r2*(GBest[i] - position[i])
    
    resultado.append(vi)
  
  return resultado

## FUNÇÃO DE VERIFICAÇÃO DE PARTICULA EM DIMENSÃO

def ParticulaInDimension(particula, feasibleBounds1, feasibleBounds2):
  
  for i in range(0, len(particula)):
    if(particula[i]<feasibleBounds1):
      particula[i] = feasibleBounds1
      
    elif(particula[i]>feasibleBounds2):
      particula[i] = feasibleBounds2
  
  return particula

## FUNÇÃO MEDIA DE VETORES

def mediaVetorGBest(vetorDeVetores):
  
  quantVetores = len(vetorDeVetores)
  tamanhoVetor = len(vetorDeVetores[0])
  vetorResultado = []
  for i in range(0,tamanhoVetor):
    soma = 0
    for j in range(0, quantVetores):
      soma += vetorDeVetores[j][i]
    soma/=quantVetores
    vetorResultado.append(soma)
    
  return vetorResultado

In [5]:
def criadorDeParticulas(numParticulas, dimensoes, feasibleBounds1, feasibleBounds2):
  particulas = []
  for i in range(0,numParticulas):
    position = []
    velocity = []
    for j in range(0, dimensoes):
      position.append(uniform(feasibleBounds1,feasibleBounds2))
      #velocity.append(uniform(feasibleBounds1,feasibleBounds2))
      velocity.append(0)
    
    p = Particula(position, velocity, 999999999999, position)
    particulas.append(p)
  
  return particulas

def adicionarVizinhos(particulas):
  
  particulas[0].updateIndice(len(particulas)-1, 1)
  
  for i in range(1,(len(particulas)-1)):
    particulas[i].updateIndice(i-1, i+1)
    
  particulas[len(particulas)-1].updateIndice((len(particulas)-2),0)

In [6]:

def PSO(nParticulas, dimensoes, feasibleBounds1, feasibleBounds2, c1, c2, wMax, wMin, avaliacoes, tipoTopologia, tipoFuncao, tipoW):

  l = 0
  GBest = []
  valueGBest = 999999999999
  vetorGBest = []
  foco = randint(0, nParticulas-1)
  iteracoes = (avaliacoes/nParticulas)
  #Criando as particulas
  p = criadorDeParticulas(nParticulas, dimensoes, feasibleBounds1, feasibleBounds2)

  if tipoTopologia == 2:
    adicionarVizinhos(p)

  while l < avaliacoes:
    for i in p:
        
      if tipoFuncao == 1:
        currentValue = sphere(i.position)
        l += 1
      elif tipoFuncao == 2:
        currentValue = rastrigin(i.position)
        l += 1
      else:
        currentValue = rosenbrock(i.position)
        l += 1

      if currentValue < i.valuePBest:
        i.updateValuePBest(valuePBest=currentValue)
        i.updatePBest(i.position)
        if currentValue < valueGBest:
          valueGBest = currentValue
          GBest = i.position

    vetorGBest.append(valueGBest)
    k = 0
    for i in p:

      r1 = random()
      r2 = random()
      clerc = False
      ## TIPO DO W
      if tipoW == 1:
        w = 0.8

      elif tipoW == 2:
        #calcular o decaimento
        w = (wMax - wMin)*((iteracoes-l)/iteracoes) + wMin

      else:
        #calcular função clerc 
        t = c1 + c2
        w = 2 / abs(2 - t - math.sqrt(t**2-4*t))
        clerc = True

      ## TIPO TOPOLOGIA
      if tipoTopologia == 1:
        newVelocity = calculateVelocity(i.velocity, c1, c2, w, r1, r2, i.PBest, GBest, i.position, clerc, feasibleBounds1, feasibleBounds2)

      elif tipoTopologia == 3:
        if k == foco:
          newVelocity = calculateVelocity(i.velocity, c1, c2, w, r1, r2, i.PBest, GBest, i.position, clerc, feasibleBounds1, feasibleBounds2)
        else:
          newVelocity = calculateVelocity(i.velocity, c1, c2, w, r1, r2, i.PBest, p[foco].position, i.position, clerc, feasibleBounds1, feasibleBounds2)

      else:
        position = i.PBest
        valor = i.valuePBest
        if p[i.indiceE].valuePBest < valor:
          valor = p[i.indiceE].valuePBest
          position = p[i.indiceE].PBest
        if p[i.indiceD].valuePBest < valor:
          valor = p[i.indiceD].valuePBest
          position = p[i.indiceD].PBest
        newVelocity = calculateVelocity(i.velocity, c1, c2, w, r1, r2, i.PBest, position, i.position, clerc, feasibleBounds1, feasibleBounds2)

      ####

      i.updateVelocity(newVelocity)

      newPosition = [(a + b) for a, b in zip(i.position, i.velocity)]
      newPosition = ParticulaInDimension(newPosition, feasibleBounds1, feasibleBounds2)
      i.updatePosition(newPosition)
      k += 1
    

  return valueGBest, GBest, vetorGBest  
  

In [ ]:
### Cenário de teste, são executadas 30x para cada tipo de funcao, em cada tipo de topologia e com cada tipo de W

In [7]:
##Title Parametros
# 30 particulas
# 30 dimensoes
# 10000 interacoes
# coeficiente cognitivo c1 e coeficiente social c2 = 2,05;
results = pd.DataFrame()
resultsValue = pd.DataFrame()

nParticulas = 30
dimensoes = 30

c1 = c2 = 2.05
wMax = 0.9
wMin = 0.4
avaliacoes = 500000

##Tipo topologia - 1 - global / 2 - local / 3 - focal
##Tipo função - 1 - sphere / 2 - rastrigin / 3 - rosenbrock
## TipoW - 1 - constante / 2 - decaimento linear / 3 - fator clerc


#Vetor com os valueGBest de cada simulacao (30 posicoes)
vetorValueGBest = []
#Vetor com as posicoes GBest de cada simulacao (30 posicoes)
vetorPosicaoGBest = []


for k in range(1,4):
  tipoFuncao = k
  if k == 1:
    feasibleBounds1 = -100
    feasibleBounds2 = 100
  elif k == 2:
    feasibleBounds1 = -30
    feasibleBounds2 = 30
  else:
    feasibleBounds1 = -5.12
    feasibleBounds2 = 5.12
  for l in range(2,3):
    tipoW = l
    #Vetor com os vetores de cada valueGBest de cada iteracao
    vetorVetorGBest = []
    
    for i in range(2,3):
      tipoTopologia = i
      vetorTempVetorGBest = []
      vetorTempValueGBest = []
      for j in tqdm(range(0,30)):
        
        valueGBest, GBest, vetorGBest = PSO(nParticulas, dimensoes, feasibleBounds1, feasibleBounds2, c1, c2, wMax, wMin, avaliacoes, tipoTopologia, tipoFuncao, tipoW)
        vetorTempVetorGBest.append(vetorGBest)
        vetorTempValueGBest.append(valueGBest)
      #vetorVetorGBest.append(mediaVetorGBest(vetorTempVetorGBest))
      results[f'sim_{k}{l}{i}'] = mediaVetorGBest(vetorTempVetorGBest)
      resultsValue[f'sim_{k}{l}{i}'] = vetorTempValueGBest
      #vetorValueGBest.append(vetorTempValueGBest)
    

results.to_csv(f'pso_.csv', index=False)
resultsValue.to_csv(f'pso_2.csv', index=False)

 37%|███▋      | 11/30 [03:30<06:03, 19.11s/it]


KeyboardInterrupt: 